In [0]:
from pyspark.sql import functions as F

# Load Bronze
ratings_bronze = spark.table("workspace.movielens_bronze.ratings_bronze")
items_bronze   = spark.table("workspace.movielens_bronze.items_bronze")

# Clean ratings: convert timestamp, drop bronze metadata
ratings_silver = (
    ratings_bronze
        .filter((F.col("rating") >= 1) & (F.col("rating") <= 5))
        .withColumn("rating_ts", F.from_unixtime("rating_timestamp").cast("timestamp"))
        .withColumn("rating_date", F.to_date("rating_ts"))
        .drop("source_file", "ingestion_timestamp", "medallion_layer")
)

ratings_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.movielens_silver.ratings_silver")


# Clean items: date, remove metadata
items_silver = (
    items_bronze
        .withColumn("release_date_parsed", F.try_to_date("release_date", "dd-MMM-yyyy"))
        .drop("video_release_date")
        .drop("source_file", "ingestion_timestamp", "medallion_layer")
)

items_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.movielens_silver.items_silver")


# Join to produce the final Silver analytic table
movie_ratings_silver = (
    ratings_silver.alias("r")
        .join(
            items_silver.alias("i"),
            F.col("r.item_id") == F.col("i.movie_id"),
            "inner"
        )
)

movie_ratings_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("workspace.movielens_silver.movie_ratings_silver")


In [0]:
%sql
SELECT movie_title, rating, rating_date
FROM workspace.movielens_silver.movie_ratings_silver
LIMIT 10;


movie_title,rating,rating_date
Kolya (1996),3,1997-12-04
L.A. Confidential (1997),3,1998-04-04
Heavyweights (1994),1,1997-11-07
Legends of the Fall (1994),2,1997-11-27
Jackie Brown (1997),1,1998-02-02
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963),4,1998-01-07
"Hunt for Red October, The (1990)",2,1997-12-03
"Jungle Book, The (1994)",5,1998-04-03
Grease (1978),3,1998-02-01
"Remains of the Day, The (1993)",3,1997-12-31
